In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io

In [ ]:
from google.colab import files
data_to_load=files.upload()


Saving emails.csv to emails.csv


In [ ]:
df=pd.read_csv(io.BytesIO(data_to_load['spam_or_not_spam.csv']))
print(df)

                                                  email  label
0      date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...      0
1     martin a posted tassos papadopoulos the greek ...      0
2     man threatens explosion in moscow thursday aug...      0
3     klez the virus that won t die already the most...      0
4      in adding cream to spaghetti carbonara which ...      0
...                                                 ...    ...
2995   abc s good morning america ranks it the NUMBE...      1
2996   hyperlink hyperlink hyperlink let mortgage le...      1
2997   thank you for shopping with us gifts for all ...      1
2998   the famous ebay marketing e course learn to s...      1
2999   hello this is chinese traditional 子 件 NUMBER世...      1

[3000 rows x 2 columns]


In [ ]:
# df
df.rename(columns={'email':'Email_Subject'}, inplace = True)
df #Label = 0 = Normal Email, Label = 1 = Spam Email

,Email_Subject,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0
...,...,...
2995,abc s good morning america ranks it the NUMBE...,1
2996,hyperlink hyperlink hyperlink let mortgage le...,1
2997,thank you for shopping with us gifts for all ...,1
2998,the famous ebay marketing e course learn to s...,1


In [ ]:
df.info()
# There is a "null subject" in email subject
# Label is in Integer type, do not need to further convert
df['Email_Subject'].fillna('', inplace = True) # filled 'Null' with ''(blank)
df['Prediction'].value_counts()

NameError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
# Import the model for splitting training set & testing set
X, y = df['Email_Subject'], df['Prediction']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,  random_state=1)
# Using 8:2 portion for Train:Test

NameError: ignored

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
# Model for counting the terms frequency

count_vector = CountVectorizer(stop_words='english', lowercase = True)

# Count Vec automatically keep the data in Sparse Martrix to save memory

training_data = count_vector.fit_transform (X_train)
testing_data = count_vector.transform (X_test)

In [ ]:
f1_scores = {}
accuracy_scores = {}

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
#%%time
model = MultinomialNB()
model.fit(training_data.toarray(),y_train)
pred = model.predict(testing_data.toarray())
# prob = model.predict_proba(testing_data.toarray()) #predict the probability that it could be a spam
# accuracy_score on test set
accuracy = accuracy_score(y_test, pred)  # TEST SET accuracy of MultinomialNB
print(f'Accuracy score: {accuracy}')

# precision_score, recall_score, f1_score on test set
precision = precision_score(y_test, pred)
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

f1_scores['MultinomialNB'] = round(f1,3)
accuracy_scores['MultinomialNB'] = round(accuracy,3)
print(f'precision score: {precision}')
print(f'recall score: {recall}')
print(f'f1 score: {f1}')

Accuracy score: 0.9883333333333333
precision score: 0.99
recall score: 0.9428571428571428
f1 score: 0.9658536585365853


In [ ]:
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       495
           1       0.99      0.94      0.97       105

    accuracy                           0.99       600
   macro avg       0.99      0.97      0.98       600
weighted avg       0.99      0.99      0.99       600



In [ ]:
confusion = confusion_matrix(y_test, pred)
print(confusion)


[[494   1]
 [  6  99]]


In [ ]:
# True negative, false positive, false negative, true positive for binary classification
tn, fp, fn, tp = confusion.ravel()
print(f'TN: {tn}')
print(f'TP: {tp}')
print(f'FP: {fp}')
print(f'FN: {fn}')

TN: 494
TP: 99
FP: 1
FN: 6


In [ ]:
# Other Testing Models
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV, cross_validate, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.linear_model import LogisticRegression
#testing on other model
#%%time
# Using GridSearch CV for tunning parameters
# for LinearSVC
params = [{'C':[0.1,0.5,1,5], 'max_iter':[75000,100000]}]

clf = LinearSVC()
search_cv = GridSearchCV(clf,params,cv=5, scoring='accuracy',return_train_score=True)
search_cv.fit(training_data.toarray(),y_train)
search_cv.best_params_


{'C': 0.1, 'max_iter': 75000}

In [ ]:
%%time
# for Random Forest
params = [{'n_estimators':[400,500,600],
          'max_depth':[30,35,40]}]

clf = RandomForestClassifier()
search_cv = GridSearchCV(clf,params,cv=5, scoring='accuracy',return_train_score=True)
search_cv.fit(training_data,y_train)
search_cv.best_params_

# Wall time: ~4min 29s
# Result: {'max_depth': 40, 'n_estimators': 500}

CPU times: user 5min 26s, sys: 710 ms, total: 5min 27s
Wall time: 5min 52s


{'max_depth': 40, 'n_estimators': 500}

In [ ]:
# Randomized CV search first
params = [{'C':[0.1,0.5,1,5],
           'solver':['lbfgs', 'liblinear'],
          'max_iter':[5000,7000,10000]
          }]
clf = LogisticRegression()
random_cv = RandomizedSearchCV(clf,params,cv=5, n_iter=24,scoring='accuracy',return_train_score=True)
random_cv.fit(training_data,y_train)
random_cv.best_params_
#{'solver': 'lbfgs', 'max_iter': 5000, 'C': 5}

{'solver': 'lbfgs', 'max_iter': 5000, 'C': 5}

In [ ]:
%%time
# for SVC
params = [{'C':[0.5,1.0,1.5],
           'kernel':['linear','poly','rbf','sigmoid']}]

clf = SVC()
search_cv = GridSearchCV(clf,params,cv=5, scoring='accuracy',return_train_score=True)
search_cv.fit(training_data,y_train)
search_cv.best_params_

CPU times: user 2min, sys: 204 ms, total: 2min
Wall time: 2min


{'C': 0.5, 'kernel': 'linear'}

In [ ]:
%%time
# for Logistic Regression
params = [{'C':[1,5],
           'solver':['lbfgs','liblinear'],
          'max_iter':[5000,7000]
          }]

clf = LogisticRegression()
search_cv = GridSearchCV(clf,params,cv=5, scoring='accuracy',return_train_score=True)
search_cv.fit(training_data,y_train)
search_cv.best_params_

CPU times: user 16.9 s, sys: 20.6 s, total: 37.6 s
Wall time: 22.9 s


{'C': 5, 'max_iter': 5000, 'solver': 'lbfgs'}

In [ ]:
#fitting into different classifier
%%time

model = LinearSVC(C=0.1,max_iter=75000)
model.fit(training_data.toarray(),y_train)
pred = model.predict(testing_data.toarray())
# Accuracy score
accuracy = accuracy_score(y_test, pred)
precision = precision_score(y_test, pred)
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

# saving accuracy score, f1 score and time
f1_scores['LinearSVC'] = round(f1,3)
accuracy_scores['LinearSVC'] = round(accuracy,3)

print('Accuracy score:' + str(accuracy))
print('precision score:' + str(precision))
print(f'recall score: {recall}')
print(f'f1 score: {f1}')

Accuracy score:0.9883333333333333
precision score:0.99
recall score: 0.9428571428571428
f1 score: 0.9658536585365853
CPU times: user 683 ms, sys: 401 ms, total: 1.08 s
Wall time: 1.08 s


In [ ]:
%%time
model = RandomForestClassifier(n_estimators=600, max_depth=40)
model.fit(training_data.toarray(),y_train)
pred = model.predict(testing_data.toarray())

# Accuracy score
accuracy = accuracy_score(y_test, pred)
precision = precision_score(y_test, pred)
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

f1_scores['Random_Forest'] = round(f1,3)
accuracy_scores['Random_Forest'] = round(accuracy,3)

print(f'Accuracy score: {accuracy}')
print(f'precision score: {precision}')
print(f'recall score: {recall}')
print(f'f1 score: {f1}')

Accuracy score: 0.9633333333333334
precision score: 0.9770114942528736
recall score: 0.8095238095238095
f1 score: 0.8854166666666666
CPU times: user 1min 2s, sys: 450 ms, total: 1min 3s
Wall time: 1min 2s


In [ ]:
%%time
model = LogisticRegression(C=5, max_iter=5000, solver='lbfgs')
model.fit(training_data.toarray(),y_train)
pred = model.predict(testing_data.toarray())

# Accuracy score
accuracy = accuracy_score(y_test, pred)
precision = precision_score(y_test, pred)
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

f1_scores['Logistic_Regression'] = round(f1,3)
accuracy_scores['Logistic_Regression'] = round(accuracy,3)

print(f'Accuracy score: {accuracy}')
print(f'precision score: {precision}')
print(f'recall score: {recall}')
print(f'f1 score: {f1}')

Accuracy score: 0.9866666666666667
precision score: 0.98989898989899
recall score: 0.9333333333333333
f1 score: 0.9607843137254903
CPU times: user 18.1 s, sys: 11.3 s, total: 29.4 s
Wall time: 16.5 s


In [ ]:
%%time
model = SVC(C=0.5, kernel='linear')
model.fit(training_data.toarray(),y_train)
pred = model.predict(testing_data.toarray())
# Accuracy score
accuracy = accuracy_score(y_test, pred)
precision = precision_score(y_test, pred)
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

f1_scores['SVC'] = round(f1,3)
accuracy_scores['SVC'] = round(accuracy,3)

print(f'Accuracy score: {accuracy}')
print(f'precision score: {precision}')
print(f'recall score: {recall}')
print(f'f1 score: {f1}')

Accuracy score: 0.985
precision score: 0.9897959183673469
recall score: 0.9238095238095239
f1 score: 0.9556650246305419
CPU times: user 40.9 s, sys: 7.52 s, total: 48.4 s
Wall time: 27.2 s


In [ ]:
#compairng the result
print(f1_scores)
print(accuracy_scores)

{'MultinomialNB': 0.966, 'LinearSVC': 0.966, 'Random_Forest': 0.885, 'Logistic_Regression': 0.961, 'SVC': 0.956}
{'MultinomialNB': 0.988, 'LinearSVC': 0.988, 'Random_Forest': 0.963, 'Logistic_Regression': 0.987, 'SVC': 0.985}


In [ ]:
# turning the scores to df

df_f1_scores = pd.DataFrame(list(f1_scores.items()))
df_f1_scores.rename(columns = {0:'Models', 1:'F1 scores'}, inplace=True)

df_accuracy = pd.DataFrame(list(accuracy_scores.items()))
df_accuracy.rename(columns = {0:'Model', 1:'Accuracy scores'}, inplace=True)

#joining both df

df_model_performance = pd.concat([df_f1_scores,df_accuracy],axis=1,join='inner')
df_model_performance.drop(['Model'],axis=1,inplace=True)
print(df_model_performance)

                Models  F1 scores  Accuracy scores
0        MultinomialNB      0.966            0.988
1            LinearSVC      0.966            0.988
2        Random_Forest      0.885            0.963
3  Logistic_Regression      0.961            0.987
4                  SVC      0.956            0.985


In [ ]:
# plotting the graph based on the f1 and accuracy score
import plotly.graph_objects as go

fig = go.Figure(data=[go.Bar(name='F1 socre',x=df_model_performance['Models'],
                             y=df_model_performance['F1 scores'], text=df_model_performance['F1 scores']),
                       go.Bar(name='Accuracy',x=df_model_performance['Models'],
                          y=df_model_performance['Accuracy scores'], text=df_model_performance['Accuracy scores'])])


fig.update_layout(barmode='group')
fig.show()